In [4]:
import sys; sys.path.append('../data')
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, GINConv, global_max_pool

from chem_preprocess import get_graph_dataset 

ImportError: cannot import name 'HeteroData' from 'torch_geometric.data' (/Users/owenqueen/opt/anaconda3/lib/python3.8/site-packages/torch_geometric/data/__init__.py)

In [ ]:
# Get graph dataloader:
energies = pd.read_excel('../data/initial_data/energies.xlsx', index_col = 0)
train_loader, test_loader = get_graph_dataset(
    '../data/initial_data/STRUCTS',
    Y = energies.iloc[:,0])

In [ ]:
# Define model
class GCN(torch.nn.Module):
    def __init__(self, hidden_channels, input_feat):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(input_feat, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.lin = torch.nn.Linear(hidden_channels, 1)

    def forward(self, x, edge_index, batch):
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = F.dropout(x, p=0.25, training=self.training)
        x = self.conv2(x, edge_index)
        x = global_max_pool(x, batch)
        return self.lin(x)

In [ ]:
model = GCN(hidden_channels = 64, input_feat = 2

optimizer = torch.optim.Adam(model.parameters())
criterion = torch.nn.MSELoss()

# Define training and testing loops
def train(model, optimizer, criterion, data_loader):
    model.train()
    for data in data_loader:
        out = model(data.x, data.edge_index, data.batch)
        loss = criterion(out, data.y)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()